# Proyecto Final: Integración de 3 o más cuerpos

- Carlos Daniel López Mimenza (**@SirSilverCharles**)
- Daniel Martínez Urrieta (**@danmarurr**)
- María Fernanda Pérez Ramírez (**@FernandaPerez**)

Para el problema de $N$ cuerpos, consideremos las ecuaciones de movimiento para cada una ellas (con la posición $\vec{q}_i$, masa $m_i$), suponiendo que las partículas se encuentran en el mismo plano (2D):

\begin{align}
    m_i \frac{{\rm d}^2\vec{q}_i}{{\rm d}t^2} = \sum_{i \neq j} \frac{Gm_im_j}{||\vec{q}_i - \vec{q}_j||^3}(\vec{q}_i - \vec{q}_j)
\end{align}

## Caso más simple: Tres cuerpos.

Para éste caso, tendremos que las ecuaciones que describen las trayectorias de las partículas son:

\begin{align}
    \ddot{\mathbf{x}}_1 &= - \frac{Gm_2}{(x_1 - x_2)^3}(\mathbf{x}_1 - \mathbf{x}_2) - \frac{Gm_3}{(x_1 - x_3)^3}(\mathbf{x}_1 - \mathbf{x}_3)\\
    \ddot{\mathbf{x}}_2 &= - \frac{Gm_3}{(x_2 - x_3)^3}(\mathbf{x}_2 - \mathbf{x}_3) - \frac{Gm_1}{(x_2 - x_1)^3}(\mathbf{x}_2 - \mathbf{x}_1)\\
    \ddot{\mathbf{x}}_3 &= - \frac{Gm_1}{(x_3 - x_1)^3}(\mathbf{x}_3 - \mathbf{x}_1) - \frac{Gm_2}{(x_3 - x_2)^3}(\mathbf{x}_3 - \mathbf{x}_2)
\end{align}

Para tomar sólo ecuaciones de primer orden (y así usar Taylor) implementamos además que:
\begin{align}
    \dot{\mathbf{v}}_j = \ddot{\mathbf{x}}_j \ \ \Rightarrow \dot{\mathbf{x}}_j = \mathbf{v}_j
\end{align}